# Evaluación de respuesta

In [2]:
import pandas as pd
import sqlite3 as sql
import json

from IPython.display import Markdown, display

In [3]:
# Conexión a la base de datos log_publico.db
conn = sql.connect('log_publico.db')

# Ejemplo: leer todas las tablas disponibles
query = "SELECT * FROM interacciones;"
df = pd.read_sql(query, conn)
df.head()

,id,timestamp,pregunta,clasificacion_json,dato_recuperado,respuesta_final
0,1,2025-07-28T18:27:22.818892,Estamos ante el pico más alto en la recaudación?,"{""intencion"": ""estadistica"", ""periodo"": ""Ultim...",{'último periodo': detapart\nRecursos Corrient...,No se puede afirmar categóricamente que se est...
1,2,2025-07-28T18:29:12.995109,Me puedes decir cual es la recaudación en ingr...,"{""intencion"": ""estadistica"", ""periodo"": ""Ultim...",{'último periodo': detapart\nRecursos Corrient...,La recaudación bruta en Ingresos Brutos hasta ...
2,3,2025-07-28T18:31:44.224299,Me puedes decir cual es la recaudación en ingr...,"{""intencion"": ""estadistica"", ""periodo"": ""Ultim...",{'último periodo': detapart\nRecursos Corrient...,Error al generar la respuesta de la IA: 429 Yo...
3,4,2025-07-28T18:31:44.927741,En el mes de marzo de este año tenemos una mej...,"{""intencion"": ""estadistica"", ""periodo"": ""Espec...",{'último periodo': detapart\nRecursos Corrient...,Error al generar la respuesta de la IA: 429 Yo...
4,5,2025-07-28T19:06:48.980482,Me puedes decir cual es la recaudación en ingr...,"{""intencion"": ""estadistica"", ""periodo"": ""Ultim...",{'último periodo': detapart\nRecursos Corrient...,Error al generar la respuesta de la IA: 429 Yo...


In [4]:
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS evaluaciones (
        id INTEGER PRIMARY KEY ,
        intencion_pred TEXT,
        periodo_pred TEXT,
        estadistica_pred TEXT,
        intencion_real TEXT,
        periodo_real TEXT,
        estadistica_real TEXT,
        veracidad INTEGER,
        entendible INTEGER,
        profundidad INTEGER,
        CEFIM INTEGER,
        coherencia INTEGER,
        dato_falso INTEGER               
    )
    """)
conn.commit()
conn.close()


In [5]:
lista = []

for i in range(len(df)):
    dicc = json.loads(df.iloc[i]['clasificacion_json'])
    lista.append(dicc)
    

In [6]:
df2 = pd.concat([df[['id','pregunta', 'respuesta_final']],pd.DataFrame(lista)],axis = 1)
df2

,id,pregunta,respuesta_final,intencion,periodo,estadistica
0,1,Estamos ante el pico más alto en la recaudación?,No se puede afirmar categóricamente que se est...,estadistica,Ultimo,rec_actual
1,2,Me puedes decir cual es la recaudación en ingr...,La recaudación bruta en Ingresos Brutos hasta ...,estadistica,Ultimo,rec_actual
2,3,Me puedes decir cual es la recaudación en ingr...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,rec_actual
3,4,En el mes de marzo de este año tenemos una mej...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Específico,rec_actual
4,5,Me puedes decir cual es la recaudación en ingr...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,rec_actual
...,...,...,...,...,...,...
107,108,¿Se está trabajando en un plan de ordenamiento...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,rec_actual
108,109,¿Qué se está haciendo para fomentar la formali...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Histórico,evolución
109,110,¿Cómo se va a garantizar la transparencia en l...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,variacion
110,111,¿Se ha pensado en crear un fondo específico co...,"Claro, con gusto analizaré la pregunta del usu...",estadistica,Histórico,evolución


In [7]:

def mostrar_pregunta(numero):
    numero -=1
    d = df2.iloc[numero]

    display(Markdown(d['pregunta']))
    print(d['periodo'], d['estadistica'])
    display(Markdown(d['respuesta_final']))   


In [8]:
mostrar_pregunta(1)

Estamos ante el pico más alto en la recaudación?

Ultimo rec_actual


No se puede afirmar categóricamente que se esté ante el pico más alto de recaudación considerando solo los datos provistos.  Si bien la recaudación de Recursos Corrientes en el último periodo (marzo 2025) muestra 335.791,51,  y el acumulado anual de 2025 alcanza los 1.019.139,26, estos valores deben ser contextualizados.

El análisis de la media móvil a cinco años (2021-2025) revela una cierta volatilidad.  Mientras que 2022 y 2023 muestran valores superiores a los de 2025,  la media móvil suaviza las fluctuaciones.  Para determinar si se trata del pico más alto, se requiere una análisis más profundo considerando la tendencia histórica a largo plazo y factores externos que podrían estar incidiendo en la recaudación, información que no se proporciona en los datos.  Se recomienda consultar los informes completos del CEFIM para una conclusión más robusta.  Los datos actuales por sí solos no permiten confirmar o refutar la hipótesis de un pico máximo.


In [9]:
df2

,id,pregunta,respuesta_final,intencion,periodo,estadistica
0,1,Estamos ante el pico más alto en la recaudación?,No se puede afirmar categóricamente que se est...,estadistica,Ultimo,rec_actual
1,2,Me puedes decir cual es la recaudación en ingr...,La recaudación bruta en Ingresos Brutos hasta ...,estadistica,Ultimo,rec_actual
2,3,Me puedes decir cual es la recaudación en ingr...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,rec_actual
3,4,En el mes de marzo de este año tenemos una mej...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Específico,rec_actual
4,5,Me puedes decir cual es la recaudación en ingr...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,rec_actual
...,...,...,...,...,...,...
107,108,¿Se está trabajando en un plan de ordenamiento...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,rec_actual
108,109,¿Qué se está haciendo para fomentar la formali...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Histórico,evolución
109,110,¿Cómo se va a garantizar la transparencia en l...,Error al generar la respuesta de la IA: 429 Yo...,estadistica,Ultimo,variacion
110,111,¿Se ha pensado en crear un fondo específico co...,"Claro, con gusto analizaré la pregunta del usu...",estadistica,Histórico,evolución


In [14]:
class evaluar_respuestas():
    def __init__(self, numero_pregunta):
        self.DB_FILE = "log_publico.db"
        self.numero = numero_pregunta
        self.intencion_real = ''
        self.periodo_real = ''
        self.estadistica_real = ''

        self.veracidad = -1 #la respuesta es verdadera
        self.entendible = -1 #la respuesta es fácil de entender
        self.prfundidad = -1 #usa bien el contexto y explica bien
        self.CEFIM = -1  #la respuesta está alineada con el CEFIM
        self.coherencia = -1 # la respueta es coherente.
        self.dato_falso = -1 #la respuesta proporciona dato falso.

    def evaluar(self):
        print("Evaluando pregunta número:", self.numero)
        mostrar_pregunta(self.numero)
        self.intencion_real = input("Intención real: ")
        self.periodo_real = input("Periodo real: ")
        self.estadistica_real = input("Estadística real: ")

        self.veracidad = int(input("¿La respuesta es verdadera? (1=Sí, 0=No): "))
        self.entendible = int(input("¿La respuesta es fácil de entender? (1=Sí, 0=No): "))
        self.prfundidad = int(input("¿Usa bien el contexto y explica bien? (4=Muy bien,3= bien, 1= mal, 0= Muy mal): "))
        self.CEFIM = int(input("¿Está alineada con el CEFIM? (4=Muy bien,3= bien,2=regular, 1= mal, 0= Muy mal): "))
        self.coherencia = int(input("¿La respuesta es coherente? (1=Sí, 0=No): "))
        self.dato_falso = int(input("¿Proporciona dato falso? (1=Sí, 0=No): "))
        self.comentario = input("Escribe un comentario: ")
        
        self.guardar_datos()
        display(self.intencion_real, self.periodo_real, self.estadistica_real,
               self.veracidad, self.entendible, self.prfundidad,
               self.CEFIM, self.coherencia, self.dato_falso)
        
    def guardar_datos(self):
        conn =  sql.connect('log_publico.db')
        cursor = conn.cursor()
        pregunta = df2.iloc[self.numero-1]

        datos = (
            self.numero,
            pregunta['intencion'],
            pregunta['periodo'],
            pregunta['estadistica'],
            self.intencion_real,
            self.periodo_real,
            self.estadistica_real,
            self.veracidad,
            self.entendible,
            self.prfundidad,
            self.CEFIM,
            self.coherencia,
            self.dato_falso,
            self.comentario
        )
        cursor.execute("""
                       INSERT INTO evaluaciones (id, intencion_pred, periodo_pred, estadistica_pred, intencion_real, periodo_real, estadistica_real, veracidad, entendible, profundidad, CEFIM, coherencia, dato_falso, comentarios)
                       VALUES (?, ?, ?, ?, ?,?,?,?,?,?,?,?,?,?)
                       
                       """,datos)
        conn.commit()
        conn.close()



In [17]:
preg = evaluar_respuestas(1)
preg.evaluar()

Evaluando pregunta número: 1


Estamos ante el pico más alto en la recaudación?

Ultimo rec_actual


No se puede afirmar categóricamente que se esté ante el pico más alto de recaudación considerando solo los datos provistos.  Si bien la recaudación de Recursos Corrientes en el último periodo (marzo 2025) muestra 335.791,51,  y el acumulado anual de 2025 alcanza los 1.019.139,26, estos valores deben ser contextualizados.

El análisis de la media móvil a cinco años (2021-2025) revela una cierta volatilidad.  Mientras que 2022 y 2023 muestran valores superiores a los de 2025,  la media móvil suaviza las fluctuaciones.  Para determinar si se trata del pico más alto, se requiere una análisis más profundo considerando la tendencia histórica a largo plazo y factores externos que podrían estar incidiendo en la recaudación, información que no se proporciona en los datos.  Se recomienda consultar los informes completos del CEFIM para una conclusión más robusta.  Los datos actuales por sí solos no permiten confirmar o refutar la hipótesis de un pico máximo.


OperationalError: database is locked

In [18]:
preg.guardar_datos()

In [9]:
mostrar_pregunta(74)

¿Por qué se destinan casi 100 mil millones de pesos al Fondo Compensador del Transporte en el presupuesto 2024, si el servicio sigue siendo deficiente según los usuarios?

Ultimo variacion


Aquí hay un análisis sobre el Fondo Compensador del Transporte en el presupuesto 2024, basado en los datos proporcionados y la perspectiva del CEFIM:

La asignación de casi 100 mil millones de pesos al Fondo Compensador del Transporte en el presupuesto 2024, a pesar de las quejas de los usuarios sobre la calidad del servicio, puede parecer contradictoria a primera vista. Sin embargo, según el CEFIM, es necesario considerar varios factores económicos y políticos.

Según el último informe de CEFIM, la disminución general de los recursos de origen nacional en un -12.61 % en 2024, en comparación con una caída más moderada de los recursos de origen provincial (-10.54%) podría ejercer presión sobre el presupuesto provincial, haciendo que el Fondo Compensador sea crucial para mantener el servicio de transporte. En particular, la fuerte caída proyectada en la Ley 25.053 Fdo. Nac. de Incentivo Docente (-93.25%) podría estar influyendo en esta necesidad.

Además, la caída significativa en los ingresos brutos (-9.37%) en 2024, que son un indicador clave de la actividad económica provincial, sugiere que el Fondo Compensador podría ser una herramienta para estimular la demanda y mantener la movilidad, especialmente para los ciudadanos de bajos ingresos.  El CEFIM ha notado previamente que los años de declive económico a menudo requieren una mayor intervención estatal para garantizar servicios esenciales.

En última instancia, la decisión de asignar una cantidad considerable de fondos al Fondo Compensador, a pesar de las deficiencias del servicio, es probablemente un reflejo de las complejas prioridades políticas y económicas que dan forma al presupuesto provincial. El CEFIM continuará monitoreando la efectividad de estos fondos y su impacto en la calidad del servicio de transporte.


In [10]:
mostrar_pregunta(34)

En el gráfico de barras, ¿se podría mostrar la dispersión de los datos de los últimos 8 años para entender qué tan volátil ha sido la recaudación?

Histórico evolución


Claro, aquí tienes un análisis sobre la dispersión de los datos de recaudación en los últimos 8 años, utilizando los datos proporcionados y el contexto del CEFIM:

Para evaluar la volatilidad de la recaudación en los últimos 8 años, podemos analizar la dispersión de los recursos corrientes tanto de origen provincial como nacional. Utilizando los datos anuales desde 2017 hasta 2024, observamos las siguientes tendencias:

- **Recursos de Origen Provincial:** Estos recursos, reflejo de la economía provincial, muestran cierta estabilidad en comparación con los recursos de origen nacional. Sin embargo, se aprecia una leve disminución en los últimos años, lo que podría indicar una desaceleración en la actividad económica provincial.

- **Recursos de Origen Nacional:** Principalmente compuestos por la coparticipación federal, estos recursos están sujetos a las políticas económicas nacionales. La dispersión en estos datos puede ser mayor debido a factores externos y decisiones gubernamentales a nivel nacional.

Para medir la dispersión, se podrían calcular estadísticas como la desviación estándar o el rango intercuartílico para cada tipo de recurso. Sin embargo, con los datos proporcionados, podemos inferir que los recursos provinciales tienden a ser menos volátiles, ya que están más directamente relacionados con la actividad económica local, mientras que los recursos nacionales pueden fluctuar más debido a factores externos.

Según el CEFIM, es crucial monitorear ambas fuentes de ingresos para entender la salud financiera de la provincia y su capacidad para afrontar los gastos presupuestarios.


In [13]:
mostrar_pregunta(54)

¿Se podría hacer un gráfico de torta que muestre la composición actual de los recursos de origen provincial?

Ultimo rec_actual


Claro, puedo ayudarte con eso.

**Análisis de la Composición de Recursos de Origen Provincial**

Según los datos más recientes disponibles (detapart), los recursos de origen provincial se componen principalmente de ingresos tributarios y no tributarios. Dentro de los tributarios, el impuesto a los Ingresos Brutos representa la mayor parte con $83,946.27, seguido por Automotor ($19,703.11), Inmobiliario ($8,276.48), y Sellos y Tasas de Justicia ($10,262.48). Los ingresos no tributarios están liderados por Regalías ($14,199.26) y Tasas Retributivas de Servicios ($5,698.96).

Para representar gráficamente esta composición, un gráfico de torta sería adecuado para visualizar la proporción de cada componente dentro del total de recursos de origen provincial, tal como fue solicitado. En este caso la porción más grande del gráfico correspondería al impuesto sobre los ingresos brutos.

Es importante recordar, como indica el CEFIM, que los recursos de origen provincial, en su mayoría, provienen del impuesto a los ingresos brutos, lo que refleja el comportamiento de la economía provincial en el periodo analizado.


In [14]:
mostrar_pregunta(107)

¿Cuál es el plan del gobierno para mejorar la calidad del transporte público, más allá de aumentar los subsidios?

Histórico rec_actual


De acuerdo con el análisis del CEFIM, más allá de los subsidios, no se pueden identificar planes concretos del gobierno para mejorar la calidad del transporte público con la información disponible. Los datos proporcionados se centran en los recursos corrientes de la provincia, tanto de origen provincial como nacional, así como en las erogaciones corrientes. Estos datos son útiles para evaluar la salud financiera general y la actividad económica, pero no ofrecen detalles sobre políticas específicas de inversión o mejora en el sector del transporte público.

El análisis de los recursos muestra que los ingresos brutos son una fuente importante de ingresos provinciales (83946.27 en el último periodo), y la coparticipación federal es fundamental para los recursos de origen nacional (142017.71 en el último periodo). Sin embargo, estos datos no especifican cómo se asignan estos fondos a diferentes sectores, incluido el transporte público. Para obtener información sobre los planes del gobierno en este ámbito, sería necesario acceder a datos presupuestarios detallados y a documentos de planificación sectorial específicos.
